In [1]:
import re
from langdetect import detect
import cPickle
import numpy as np

In [2]:
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return cPickle.load(f)

def save_obj(obj, name ):
    with open('obj/' + name + '.pkl', 'wb') as f:
        cPickle.dump(obj, f)#,  marshal.version)#, marshal.HIGHEST_PROTOCOL)

In [3]:
from langdetect import detect

class FeatureGenerator:
    
    def __init__(self, lm):
        self.lm = lm
    
    def generate_features(self, query):
        x = []
        words = re.findall(ur"(?u)\w+", query)
        x.append(len(words))# количество слов
        x.append(len(query))# количество символов
        x.append(self.lm.get_prob(words)) # вероятность такого запроса
        max_prob = -1.
        min_prob = 2.
        
        count_of_words_in_dict = 0
        for word in words:
            prob = self.lm.get_word_prob(word)
            if prob > max_prob:
                max_prob = prob
            if prob < min_prob:
                min_prob = prob
                
            if self.lm.dict.has_key(word):
                count_of_words_in_dict += 1
                
        x.append(max_prob) # максимальная вероятность слова
        x.append(min_prob) # минимальная вероятность слова
        x.append(len(words)-count_of_words_in_dict) # сколько слов нет в словаре
        
        if u"," in query or \
            u"." in query or \
            u"'" in query or \
            u";" in query or \
            u"]" in query or \
            u"[" in query or \
            u"~" in query:
            x.append(1) # есть ли "плохие" символы в запросе 
        else:
            x.append(0)
        
        try:
            lang = detect(query)
            lang = 1 if lang == 'en' else 0
        except Exception:
            lang = 0
            
        x.append(lang) # язык
        
        
        return x

In [5]:
lm = load_obj("LanguageModel")

In [6]:
X = []
y = []
fg = FeatureGenerator(lm)

with open("data/queries_all.txt") as f:
    content = f.readlines()

for line in content:

    line = line.decode("utf-8")
    line = line.lower()
    line = line[:-1]
    queries = line.split('\t')

    if len(queries) == 2:
        y.append(0)
        X.append(fg.generate_features(queries[0]))
        
        y.append(1)
        X.append(fg.generate_features(queries[1]))
        
    else:
        y.append(1)
        X.append(fg.generate_features(queries[0]))
    #if index > 0:
    #line = line[index+1:]

KeyboardInterrupt: 

In [10]:
y=y[:-1]

In [11]:
X = np.asarray(X)
y = np.asarray(y)

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
f1 = []
acc = []
kf = KFold(len(y), n_folds=4, shuffle=True)
for train_index, test_index in kf:
    y_train, y_test = y[train_index], y[test_index]
    X_train, X_test = X[train_index], X[test_index]
    
    gb = GradientBoostingClassifier(n_estimators=200, max_depth=3, loss='exponential')
    gb.fit(X_train, y_train)
    
    y_predicted = gb.predict(X_test)
    f1.append(f1_score(y_test, y_predicted, pos_label=0))
    acc.append(accuracy_score(y_test, y_predicted))
    
print sum(f1)/len(f1)
print sum(acc)/len(acc)

In [14]:
gb = GradientBoostingClassifier(n_estimators=200, loss='exponential')
gb.fit(X, y)

KeyboardInterrupt: 

In [36]:
save_obj(gb, "classifier")